# Практические задания

В hdfs в папке `/data/access_logs/big_log` лежит лог в формате

* IP-адрес пользователя (`195.206.123.39`),
* Далее идут два неиспользуемых в нашем случае поля (`-` и `-`),
* Время запроса (`[24/Sep/2015:12:32:53 +0400]`),
* Строка запроса (`"GET /id18222 HTTP/1.1"`),
* HTTP-код ответа (`200`),
* Размер ответа (`10703`),
* Реферер (источник перехода; `"http://bing.com/"`),
* Идентификационная строка браузера (User-Agent; `"Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/37.0.2062.94 Safari/537.36"`).

Созданы несколько семплов данных разного размера:
```
17.6 M   35.1 M   /data/access_logs/big_log_10000
2.5 G    5.0 G    /data/access_logs/middle_log
202.5 K  405.0 K  /data/access_logs/small_log
```

#### Пример парсинга логов

In [1]:
DATASET = "/data/access_logs/big_log_10000"

In [2]:
import re
import sys
from pyspark import SparkContext
from pyspark import SparkConf
from pyspark.sql import SparkSession
from datetime import datetime as dt

log_format = re.compile( 
    r"(?P<host>[\d\.]+)\s" 
    r"(?P<identity>\S*)\s" 
    r"(?P<user>\S*)\s"
    r"\[(?P<time>.*?)\]\s"
    r'"(?P<request>.*?)"\s'
    r"(?P<status>\d+)\s"
    r"(?P<bytes>\S*)\s"
    r'"(?P<referer>.*?)"\s'
    r'"(?P<user_agent>.*?)"\s*'
)

def parseLine(line):
    match = log_format.match(line)
    if not match:
        return ("", "", "", "", "", "", "" ,"", "")

    request = match.group('request').split()
    return (match.group('host'), match.group('time').split()[0], \
       request[0], request[1], match.group('status'), match.group('bytes'), \
        match.group('referer'), match.group('user_agent'),
        dt.strptime(match.group('time').split()[0], '%d/%b/%Y:%H:%M:%S').hour)


lines = sc.textFile(DATASET)
parsed_logs = lines.map(parseLine).cache()

Распарсили, получили RDD, закешировали.

In [3]:
parsed_logs.take(1)

[('109.105.128.100',
  '10/Dec/2015:00:00:00',
  'GET',
  '/id45574',
  '200',
  '27513',
  '-',
  'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/45.0.2454.99 Safari/537.36',
  0)]

Схема данных для DataFrame API
```
from pyspark.sql import SparkSession, Row
rows = parsed_logs.map(lambda p: Row(ip=p[0],
                               timestamp=p[1],
                               request_type=p[2],
                               request_url=p[3],
                               status=p[4],
                               bytes=p[5],
                               referer=p[6],
                               user_agent=p[7],
                               hour=p[8]))
```

### Задача 1
> Напишите программу, выводящую на экран TOP5 ip адресов, в которых содержится хотя бы одна цифра 4, с наибольшим количеством посещений.
Каждая строка результата должна содержать IP адрес и число посещений, разделенные табуляцией, строки должны быть упорядочены по числу посещений по убыванию, например:
```
195.206.123.39<TAB>40
196.206.123.40<TAB>39
191.206.123.41<TAB>38
175.206.123.42<TAB>37
195.236.123.43<TAB>36
```

In [ ]:
from pyspark.sql import SparkSession, Row
rows = parsed_logs.map(lambda p: Row(ip=p[0],
                               timestamp=p[1],
                               request_type=p[2],
                               request_url=p[3],
                               status=p[4],
                               bytes=p[5],
                               referer=p[6],
                               user_agent=p[7],
                               hour=p[8])).cache()

In [6]:
rows

PythonRDD[4] at RDD at PythonRDD.scala:53

In [12]:
ip = rows.map(lambda x: x[0]).cache()

In [13]:
ip

PythonRDD[12] at RDD at PythonRDD.scala:53

In [15]:
out = ip.reduceByKey(lambda x, y: x+y).cache()

In [ ]:
out.filter(lambda x: x != ())

In [ ]:
out.sortBy(lambda a: -a[1])

In [16]:
out.take(10)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: 
Aborting TaskSet 5.0 because task 1 (partition 1)
cannot run anywhere due to node and executor blacklist.
Most recent failure:
Lost task 8.0 in stage 5.0 (TID 39, sber-node03.atp-fivt.org, executor 12): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/cloudera/parcels/CDH-6.3.2-1.cdh6.3.2.p0.1605554/lib/spark/python/pyspark/worker.py", line 372, in main
    process()
  File "/opt/cloudera/parcels/CDH-6.3.2-1.cdh6.3.2.p0.1605554/lib/spark/python/pyspark/worker.py", line 367, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/opt/cloudera/parcels/CDH-6.3.2-1.cdh6.3.2.p0.1605554/lib/spark/python/pyspark/rdd.py", line 2499, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/opt/cloudera/parcels/CDH-6.3.2-1.cdh6.3.2.p0.1605554

### Задача 2
>  Напишите программу, выводящую на экран суммарное распределение количества посетителей по часам (для каждого часа в сутках вывести количество посетителей, пришедших в этот час). Id посетителя = ip + user_agent
Результат должен содержать час в сутках и число посетителей, разделенные табом и упорядоченные по часам. Например:
```
0<tab>10
1<tab>10
2<tab>10
…..
21<tab>30
22<tab>20
23<tab>10
```

In [ ]:
parsed_logs

###  Задача 3

> Найдите TOP100 самых активных пользователей. Пользователь определяется парой (ip, user_agent). Найдите топ 5 страниц (вида /id18222) которые они больше всего просмотрели.
Результат должен содержать относительный путь страницы (/id18222) и число кликов на ней. Отсортируйте в порядке убывания. Например:

```
/id18222<tab>10000
/id18223<tab>1000
/id18224<tab>100
/id18225<tab>10
/id18226<tab>1
```

### Литература по Spark
1. Изучаем Spark. Молниеносный анализ данных
2. Холден Кэрау. Эффективный Spark. Масштабирование и оптимизация.